In [1]:
import os
import sys
import glob
import pandas as pd

In [2]:
result_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/ID_table"
map_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable"
map_list = glob.glob("%s/*.txt" % map_dir)
print(map_list)
print(len(map_list))

['/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/m5C.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/m7G.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/Nm.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/m6A.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/m6Am.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/ClinVar.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/RNA_Editing.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/m1A.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/pQTL.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/meQTL.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/COSM

In [6]:
content_dict = {}
columns = pd.read_table([x for x in map_list if os.path.basename(x).startswith("eQTL")][0], sep="\t", na_filter = False).columns
prefix_list = [x.split("_")[0] for x in columns[1:]]
for prefix in prefix_list:
    prefix = " ".join(prefix.split("_"))
    print(prefix)
    get_all_records(prefix, map_list)

m1A
m5C
m6A
m6Am
m7G
Nm
pseudoU
RNA Editing
eQTL
meQTL
pQTL
sQTL
circQTL
ClinVar
GWAS
COSMIC
RBP
miRNA


In [4]:
def generate_id_index(col_name, in_list):
    prefix = col_name.split("_")[0]
    in_list = sorted(list(set(in_list)))
    if "NA" in in_list:
        in_list.remove("NA")
    result_file = os.path.join(result_dir, "%s.txt" % prefix)
    id_list = ["%s_ID_%s" % (prefix, str(num).zfill(len(str(len(in_list))))) for num in range(1, len(in_list)+1)]
    assert len(in_list) == len(id_list)
    new_list = ["%s\t%s\n" % (id_list[i], in_list[i]) for i in range(len(in_list))]
    with open(result_file, 'w') as fw:
        fw.write("ID\t%s\n" % col_name)
        fw.writelines(new_list)

In [16]:
# def pick_out_specific_cols(in_list, index_list):
#     result_list = []
#     for x in in_list:
#         tmp_list = []
#         for y in x.split("@"):
#             term_list = y.split("|")
#             tmp_list.append("|".join([term_list[i] for i in index_list]))
#         new_x = "@".join(tmp_list)
#         result_list.append(new_x)
#     return result_list

In [5]:
def get_all_records(prefix, map_list):
    columns = pd.read_table([x for x in map_list if "eQTL" in os.path.basename(x)][0], sep="\t").columns
    disease_dict = {"GWAS": [0,1,2,3,4,5,6,7,8], "COSMIC": [0,1,2,3,9,10,11,12], "ClinVar": [0,1,2,3,13,14,15,4]}
    col = [x for x in columns if x.startswith(prefix)][0]
    result_list, content_list = [], []
    for map_file in map_list:
        i_prefix = os.path.basename(map_file).split(".txt")[0]
        if i_prefix == prefix:
            df = pd.read_table(map_file, sep="\t", na_filter = False)
            content_list = list(set(df.iloc[:, 0].tolist()))
#             if prefix in disease_dict:
#                 content_list = pick_out_specific_cols(content_list, disease_dict[prefix])
        else:
            df = pd.read_table(map_file, sep="\t", na_filter = False)
            if col in df.columns:
                content_list = list(set(df[col].tolist()))
        result_list += content_list
    #
    modification_list = ["m1A","m5C","m6A","m7G","m6Am","RNA Editing","pseudoU","Nm"]
    if prefix in modification_list:
        generate_id_index(col, sorted(list(set(result_list))))
    else:
        tmp = sorted(list(set(result_list)))
        final_list = []
        for x in tmp:
            final_list += x.split("@")
        generate_id_index(col, sorted(list(set(final_list))))